In [5]:
from meshLoader import MeshLoader
import torch
from pytorch3d.datasets import ShapeNetCore
from pytorch3d.structures import Pointclouds
from pytorch3d.io import IO
import os
import numpy as np
import open3d as o3d
from tqdm.notebook import trange, tqdm


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
        
SHAPENET_PATH = "/home/ghandour/Dataset/ShapeNetCore.v2"
shapenet_dataset = ShapeNetCore(SHAPENET_PATH, version=2, load_textures=False)

/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 207e69af994efa9330714334794526d4 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 5bf2d7c2167755a72a9eb0f146e94477 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 8070747805908ae62a9eb0f146e94477 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory e6c22be1a39c9b62fb403c87929e1167 under synset directory

In [6]:
def resample_mesh(faces=None, vertices=None, mesh_cad=None, density=10000):
    """
    https://chrischoy.github.io/research/barycentric-coordinate-for-mesh-sampling/
    Samples point cloud on the surface of the model defined as vectices and
    faces. This function uses vectorized operations so fast at the cost of some
    memory.

    param mesh_cad: low-polygon triangle mesh in o3d.geometry.TriangleMesh
    param density: density of the point cloud per unit area
    param return_numpy: return numpy format or open3d pointcloud format
    return resampled point cloud

    Reference :
      [1] Barycentric coordinate system
      \begin{align}
        P = (1 - \sqrt{r_1})A + \sqrt{r_1} (1 - r_2) B + \sqrt{r_1} r_2 C
      \end{align}
    """

    if(mesh_cad is not None):
        faces = np.array(mesh_cad.triangles).astype(int)
        vertices = np.array(mesh_cad.vertices)

    vec_cross = np.cross(
        vertices[faces[:, 0], :] - vertices[faces[:, 2], :],
        vertices[faces[:, 1], :] - vertices[faces[:, 2], :],
    )

    face_areas = np.sqrt(np.sum(vec_cross ** 2, 1))

    n_samples = (np.sum(face_areas) * density).astype(int)
    # face_areas = face_areas / np.sum(face_areas)

    # Sample exactly n_samples. First, oversample points and remove redundant
    # Bug fix by Yangyan (yangyan.lee@gmail.com)
    n_samples_per_face = np.ceil(density * face_areas).astype(int)
    floor_num = np.sum(n_samples_per_face) - n_samples
    if floor_num > 0:
        indices = np.where(n_samples_per_face > 0)[0]
        floor_indices = np.random.choice(indices, floor_num, replace=True)
        n_samples_per_face[floor_indices] -= 1

    n_samples = np.sum(n_samples_per_face)

    # Create a vector that contains the face indices
    sample_face_idx = np.zeros((n_samples,), dtype=int)
    acc = 0
    for face_idx, _n_sample in enumerate(n_samples_per_face):
        sample_face_idx[acc : acc + _n_sample] = face_idx
        acc += _n_sample

    r = np.random.rand(n_samples, 2)
    A = vertices[faces[sample_face_idx, 0], :]
    B = vertices[faces[sample_face_idx, 1], :]
    C = vertices[faces[sample_face_idx, 2], :]

    P = (
        (1 - np.sqrt(r[:, 0:1])) * A
        + np.sqrt(r[:, 0:1]) * (1 - r[:, 1:]) * B
        + np.sqrt(r[:, 0:1]) * r[:, 1:] * C
    )

    return P

In [7]:
def save_dataset(verts, label, model_id):
    if(len(verts)==0):
        assert "Verts cannot be 0"
    output_dir = "/home/ghandour/Dataset/ShapeNet_PCD"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    label_folder = os.path.join(output_dir, label)
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(verts)
    pcd.estimate_normals()
    o3d.io.write_point_cloud(label_folder+"/"+model_id+".ply",pcd)
    
    


In [8]:
for model in tqdm(shapenet_dataset, ascii=True, desc="Converting Model to PCD"):
    verts, faces, label, model_id = model["verts"].numpy(), model["faces"].numpy(), model["label"], model["model_id"]
    sampled_verts = resample_mesh(faces, verts, density=10000)
    save_dataset(verts=sampled_verts, label=label, model_id=model_id)


Converting Model to PCD:   0%|          | 0/52472 [00:00<?, ?it/s]